In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [2]:
def scrape_rifle_ammo_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    products = soup.find_all("div", class_="pdp-link")
    links = []
    for i in products:
        aLink = re.findall("(?P<url>https?://[^\s]+)", str(i.find_all("a")).replace('"',""))
        links.append(aLink[0])
    return links



In [3]:
rifleAmmoByCaliberLink = "https://www.cheaperthandirt.com/shop-by?cgid=79&searchBy=Caliber"

def scrape_rifle_ammo(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")


    # get caliber number set
    caliberNumbers = []
    x = soup.find_all('div', class_="refinements")
    for i in x:
        cn = i.find_all("div", {"id": True})
        for j in cn:
            caliberNumbers.append((j["id"]))

    productLink = []
    
    # places links to products by caliber in respective section for caliber in a DF
    otherSections = soup.find_all("div", class_="brand-link-list")
    for i in otherSections:
        productLinkSection = []
        for j in i.find_all("li"):
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j.find_all("a")))
            productLinkSection.append(str(aLink[0]).replace('">', ""))
        productLink.append(productLinkSection)
    
    linksDF = pd.DataFrame(
        {"number": caliberNumbers, "section": productLink}
    )
    

    sectionedProducts = []
    for i in linksDF["section"]:
        productDF = []
        for j in i:
            detailLink = scrape_rifle_ammo_helper(j)
            productDF.append(detailLink)
        sectionedProducts.append(productDF)

    
    return sectionedProducts


       




In [4]:
sectioned_data=scrape_rifle_ammo(rifleAmmoByCaliberLink)

In [5]:
def detail_rifle_ammo_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }

    productDict = {"name":"", "price":"", "availability": ""}
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    availabilityBool = soup.find_all("script")
    soup_name = (soup.find_all("h1", class_="product-name"))
    soup_price = soup.find_all("span", {"class": "value", "itemprop": "price", "content": True })
    name = soup_name[0].text
    price = soup_price[0]["content"]
    productDict["name"] = name
    productDict["price"] = price
    

    
    for script in availabilityBool[7]:
        if "Out of Stock" in str(script):
            productDict["availability"] = "out of stock"
        else:
            productDict["availability"] = "in stock"

    return productDict
        

        
        

In [6]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice

In [8]:
"""
check if item is available by checking the form tag.
disabled should be True if its available
"""
#     # f(x) = E(|| Y - g(x)) || ^2  | x)
#     # E(Y - f(x)|) ^2 ≤ E(Y-g(x) |x)^2  -> Y is an error term
arrayOfDicts = []
for i in sectioned_data:
    for j in i:
        for k in j:
            try:
                product = detail_rifle_ammo_helper(str(k))
                print(product)
            except:
                try:
                    
                    anotherproduct = detail_rifle_ammo_helper(str(k))
                    
                except:
                    print(str(k))
    print("=========================================")



    

{'name': 'Armscor USA .22-250 Rem Ammunition 20 Rounds PT 55 Grain', 'price': '34.89', 'availability': 'in stock'}
{'name': 'Prvi Partizan .22-250 Remington Ammunition 20 Rounds SP 55 Grains', 'price': '29.89', 'availability': 'in stock'}
{'name': 'PMC Bronze .223 Rem Ammunition 20 Rounds 55 Grain FMJBT 2900 fps', 'price': '14.89', 'availability': 'in stock'}
{'name': 'American Quality .223 Remington Ammunition 250 Rounds FMJ 55 Grains F N22355VP250', 'price': 'null', 'availability': 'out of stock'}
{'name': 'TulAmmo .223 Remington Ammunition 20 Rounds 55 Grain Zinc FMJ 3241fps', 'price': '14.89', 'availability': 'in stock'}
{'name': 'Winchester .223 Remington Ammunition 55 Grain Full Metal Jacket 20 Rounds 3240 fps W223', 'price': '19.89', 'availability': 'in stock'}
{'name': 'BVAC .223 Rem. Ammunition 500 Bulk Rounds Reloaded FMJ 55 Grains R22355VP500', 'price': '399.89', 'availability': 'in stock'}
{'name': 'Armscor USA .223 Remington Ammunition 20 Rounds 55 Grain FMJ 3050 fps', 'pr